# Loading large datasets

In the previous notebook, we read the the whole taxifare .csv files into memory, specifically a Pandas dataframe, before invoking `tf.data.from_tensor_slices` from the tf.data API. We could get away with this because it was a small sample of the dataset, but on the full taxifare dataset this wouldn't be feasible.

In this notebook we demonstrate how to read .csv files directly from disk, one batch at a time, using `tf.data.TextLineDataset`

In [1]:
import tensorflow as tf
import shutil
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


## 1) Input Function Reading from CSV

We define `read_dataset()` which given a csv file path returns a `tf.data.Dataset` in which each row represents a (features,label) in the Estimator API required format 
- features: A python dictionary. Each key is a feature column name and its value is the tensor containing the data for that feature
- label: A Tensor containing the labels

We then invoke `read_dataset()` function from within the `train_input_fn()` and `eval_input_fn()`. The remaining code is as before.

In [2]:
CSV_COLUMN_NAMES = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
CSV_DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(csv_path):
    def _parse_row(row):
        # Decode the CSV row into list of TF tensors
        fields = tf.decode_csv(row, record_defaults=CSV_DEFAULTS)

        # Pack the result into a dictionary
        features = dict(zip(CSV_COLUMN_NAMES, fields))
        features.pop('key') # discard, not a real feature
        
        # Separate the label from the features
        label = features.pop('fare_amount') # remove label from features and store

        return features, label
    
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path)

    # Note:
    # If our raw data was sharded across several csv files, we would replace above with:
    # dataset = tf.data.Dataset.list_files(csv_pattern) # (i.e. data_file_*.csv)
    # dataset = dataset.flat_map(tf.data.TextLineDataset) 
    
    # Parse each CSV row into correct (features,label) format for Estimator API
    dataset = dataset.map(_parse_row)
    
    return dataset

def train_input_fn(csv_path, batch_size=128):
    #1. Convert CSV into tf.data.Dataset  with (features,label) format
    dataset = read_dataset(csv_path)
      
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
   
    return dataset

def eval_input_fn(csv_path, batch_size=128):
    #1. Convert CSV into tf.data.Dataset  with (features,label) format
    dataset = read_dataset(csv_path)

    #2.Batch the examples.
    dataset = dataset.batch(batch_size)
   
    return dataset

## 2) Create Feature Columns

Same as before

In [3]:
FEATURE_NAMES = CSV_COLUMN_NAMES[1:len(CSV_COLUMN_NAMES) - 1] # all but first and last columns

feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURE_NAMES]
feature_cols

[_NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='passengers', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## 3) Choose Estimator 

Same as before

In [10]:
OUTDIR = 'taxi_trained'

model = tf.estimator.LinearRegressor(
    feature_columns=feature_cols,
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed=1) # for reproducibility
)

INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_model_dir': 'taxi_trained', '_service': None, '_save_summary_steps': 100, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_train_distribute': None, '_master': '', '_num_worker_replicas': 1, '_eval_distribute': None, '_experimental_distribute': None, '_device_fn': None, '_task_type': 'worker', '_task_id': 0, '_protocol': None, '_evaluation_master': '', '_is_chief': True, '_save_checkpoints_steps': None, '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f100e61a780>}


## 4) Train
Same as before

In [11]:
%%time
tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model.train(
    input_fn=lambda:train_input_fn('./taxi-train.csv'),
    steps=500
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 25119.68, step = 1
INFO:tensorflow:global_step/sec: 24.5533
INFO:tensorflow:loss = 8162.833, step = 101 (4.077 sec)
INFO:tensorflow:global_step/sec: 29.221
INFO:tensorflow:loss = 9615.658, step = 201 (3.421 sec)
INFO:tensorflow:global_step/sec: 33.1332
INFO:tensorflow:loss = 6763.669, step = 301 (3.017 sec)
INFO:tensorflow:global_step/sec: 31.7854
INFO:tensorflow:loss = 18262.418, step = 401 (3.145 sec)
INFO:tensorflow:Saving checkpoints for 500 into taxi_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 9201.275.
CPU times: user 11.5 s, sys: 8.75 s, total: 20.2 s
Wall time: 18.5 s


## 5) Evaluate

Same as before

In [6]:
metrics = model.evaluate(input_fn=lambda:eval_input_fn('./taxi-valid.csv'))
print('RMSE on dataset = {}'.format(metrics['average_loss']**.5))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-21-16:36:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-21-16:36:02
INFO:tensorflow:Saving dict for global step 500: average_loss = 108.9577, global_step = 500, label/mean = 11.666427, loss = 12958.185, prediction/mean = 11.301861
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: taxi_trained/model.ckpt-500
RMSE on dataset = 10.438280636039575


## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Unlike in the challenge exercise for b_estimator.ipynb, assume that your measurements of r, h and V are all rounded off to the nearest 0.1. Simulate the necessary training dataset. This time, you will need a lot more data to get a good predictor.

Hint (highlight to see):
<p style='color:white'>
Create random values for r and h and compute V. Then, round off r, h and V (i.e., the volume is computed from the true value of r and h; it's only your measurement that is rounded off). Your dataset will consist of the round values of r, h and V. Do this for both the training and evaluation datasets.
</p>

Now modify the "noise" so that instead of just rounding off the value, there is up to a 10% error (uniformly distributed) in the measurement followed by rounding off.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License